# Gradient Descent Revisited

This notebook revisits inference for a ___linear model___ using gradient descent.

Our focus will be using `pytorch` to make the implementation more parsimonious:
1. Generate data for a linear model
1. Review gradient descent, implement inference in `numpy`
1. Implement inference using PyTorch's autograd
1. Leverage `torch.nn` module to simplify the implementation

In [1]:
import numpy as np
import torch

In [2]:
def print_result(x, label):
    x = [xi.item() if torch.is_tensor(xi) else xi for xi in x]
    print(
        f"{label+':':9s} y = {x[0]:.2f} + {x[1]:.2f} x_1 + {x[2]:.2f} x_2 + {x[3]:.2f} x_3"
    )

## Data

Our data generating process is a linear model

$
\quad \mathbf y = a + b\,\mathbf x_1 + c\,\mathbf x_2 + d\,\mathbf x_3 + \mathbf\varepsilon
$

We'll set

- $a = 3$
- $b = -2$
- $c = 1$
- $d = -1$
- $\varepsilon \sim \mathcal{N}(\mu_{\varepsilon}, \sigma_{\varepsilon})$ with $\mu_{\varepsilon}=0$ and $\sigma_{\varepsilon}=1$


In [3]:
np.random.seed(501)

N = 1_000

at = 3
bt = -2
ct = 1
dt = -1

sigma_err = 1

err = np.random.normal(0, sigma_err, N)
x1 = np.random.normal(2, 1, N)
x2 = np.random.normal(3, 1, N)
x3 = np.random.normal(1, 1, N)
y = at + bt * x1 + ct * x2 + dt * x3 + err

## Manual implementation with `numpy`

1. Manually implement loss function
1. Manually calculate gradient
1. Manually update parameters

In [4]:
torch.manual_seed(501)
np.random.seed(501)

In [5]:
# define learning rate
learning_rate = 1e-3
n_epochs = 100_000

# random starting values
am, bm, cm, dm = np.random.uniform(-1, 1, 4)

# training: looping over epochs
for e in range(n_epochs):

    y_pred = am + bm * x1 + cm * x2 + dm * x3

    loss = np.mean((y_pred - y) ** 2 / 2)

    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss:.6f}")

    grad_y_pred = y_pred - y

    grad_a = (grad_y_pred * 1).mean()
    grad_b = (grad_y_pred * x1).mean()
    grad_c = (grad_y_pred * x2).mean()
    grad_d = (grad_y_pred * x3).mean()

    am -= learning_rate * grad_a  # am = am - learning_rate * grad_a
    bm -= learning_rate * grad_b
    cm -= learning_rate * grad_c
    dm -= learning_rate * grad_d

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")

    9999 | loss = 0.619665
   19999 | loss = 0.537811
   29999 | loss = 0.513246
   39999 | loss = 0.505873
   49999 | loss = 0.503661
   59999 | loss = 0.502997
   69999 | loss = 0.502798
   79999 | loss = 0.502738
   89999 | loss = 0.502720
   99999 | loss = 0.502714

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 3.19 + -2.03 x_1 + 0.94 x_2 + -0.97 x_3


## Use `autograd` from `PyTorch`

1. Manually implement loss function
1. Use `torch` gradients
1. Manually update parameters

In [6]:
torch.manual_seed(501)
np.random.seed(501)

### Some first steps

#### Turn `numpy` arrays into `torch` tensors

In [7]:
x1t = torch.tensor(x1)
x2t = torch.tensor(x2)
x3t = torch.tensor(x3)
yt = torch.tensor(y)

#### Initialize coefficients

In [8]:
a = torch.randn((), requires_grad=True)
b = torch.randn((), requires_grad=True)
c = torch.randn((), requires_grad=True)
d = torch.randn((), requires_grad=True)

#### Make prediction

In [9]:
yt_pred = a + b * x1t + c * x2t + d * x3t
yt_pred[:10]

tensor([-0.2067, -3.6361, -5.1370, -4.8801, -0.3303, -0.8349, -5.3536, -2.8600,
        -0.6940, -3.4232], dtype=torch.float64, grad_fn=<SliceBackward0>)

#### Calculate loss

In [10]:
loss = torch.mean((yt_pred - yt).pow(2) / 2)
loss

tensor(13.8336, dtype=torch.float64, grad_fn=<MeanBackward0>)

#### Backward pass and gradients

In [11]:
loss.backward()

In [12]:
a.grad.item(), (yt_pred - yt).mean().item()

(-3.3725762367248535, -3.372576336569959)

In [13]:
b.grad.item(), ((yt_pred - yt) * x1t).mean().item()

(-4.728719234466553, -4.728719281632131)

In [14]:
c.grad.item(), ((yt_pred - yt) * x2t).mean().item()

(-12.700126647949219, -12.700126466714602)

In [15]:
d.grad.item(), ((yt_pred - yt) * x3t).mean().item()

(-1.2989070415496826, -1.2989070407896441)

### Putting it all together

In [16]:
torch.manual_seed(501)
np.random.seed(501)

In [17]:
learning_rate = 1e-3
n_epochs = 100_000

ap = torch.randn((), requires_grad=True)
bp = torch.randn((), requires_grad=True)
cp = torch.randn((), requires_grad=True)
dp = torch.randn((), requires_grad=True)

for e in range(n_epochs):

    # forward
    yt_pred = ap + bp * x1t + cp * x2t + dp * x3t
    loss = torch.mean((yt_pred - yt).pow(2) / 2)
    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss.item():.6f}")

    # backward
    loss.backward()

    with torch.no_grad():
        ap -= learning_rate * ap.grad
        bp -= learning_rate * bp.grad
        cp -= learning_rate * cp.grad
        dp -= learning_rate * dp.grad

        ap.grad = None
        bp.grad = None
        cp.grad = None
        dp.grad = None

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")
print_result([ap, bp, cp, dp], "PyTorch")

    9999 | loss = 0.541866
   19999 | loss = 0.514465
   29999 | loss = 0.506240
   39999 | loss = 0.503772
   49999 | loss = 0.503031
   59999 | loss = 0.502808
   69999 | loss = 0.502741
   79999 | loss = 0.502721
   89999 | loss = 0.502715
   99999 | loss = 0.502713

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 3.19 + -2.03 x_1 + 0.94 x_2 + -0.97 x_3
PyTorch:  y = 3.19 + -2.03 x_1 + 0.94 x_2 + -0.97 x_3


## Using `torch.nn`

1. Use `torch` loss function
1. Use `torch` gradients
1. Use `torch` parameter update

In [18]:
torch.manual_seed(501)
np.random.seed(501)

In [19]:
xt = torch.tensor(np.column_stack([x1, x2, x3]).astype(np.float32))
yt = torch.tensor(y.astype(np.float32))

In [20]:
model = torch.nn.Sequential(torch.nn.Linear(3, 1), torch.nn.Flatten(0, 1))

In [21]:
learning_rate = 1e-3
n_epochs = 100_000

loss_fn = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for e in range(n_epochs):

    # forward
    yt_pred = model(xt)
    loss = loss_fn(yt_pred, yt) / 2

    if e % 10_000 == 9_999:
        print(f"{e:8d} | loss = {loss.item():.6f}")

    optimizer.zero_grad()
    # backward
    loss.backward()

    # udpate
    optimizer.step()

print(f"\nResult")
print_result([at, bt, ct, dt], "True")
print_result([am, bm, cm, dm], "Manual")
print_result([ap, bp, cp, dp], "PyTorch")
print_result(torch.cat([model[0].bias, model[0].weight.flatten()]), "PyTorch2")

    9999 | loss = 0.563865
   19999 | loss = 0.521067
   29999 | loss = 0.508222
   39999 | loss = 0.504367
   49999 | loss = 0.503209
   59999 | loss = 0.502862
   69999 | loss = 0.502757
   79999 | loss = 0.502726
   89999 | loss = 0.502716
   99999 | loss = 0.502713

Result
True:     y = 3.00 + -2.00 x_1 + 1.00 x_2 + -1.00 x_3
Manual:   y = 3.19 + -2.03 x_1 + 0.94 x_2 + -0.97 x_3
PyTorch:  y = 3.19 + -2.03 x_1 + 0.94 x_2 + -0.97 x_3
PyTorch2: y = 3.19 + -2.03 x_1 + 0.94 x_2 + -0.97 x_3


<br>
<br>
<b>Learning from Big Data</b> <br>
Sebastian Gabel <br>

<br>

Inspired by this [PyTorch Tutorial](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)